# Basic Neural Net & Library

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
plt.style.use('ggplot')

import torch
from   torch import nn
from torch.utils.data import Dataset,DataLoader

# Sklearn Iris Dataset

In [2]:
from sklearn import datasets

class IrisDataset(Dataset):

    # data loading
    def __init__(self):
        iris = datasets.load_iris()
        feature   = pd.DataFrame(iris.data, columns=iris.feature_names)
        target    = pd.DataFrame(iris.target, columns=['target'])
        iris_data = pd.concat([target, feature], axis=1)
        # Data type change and flatten targets
        
        self.x = torch.from_numpy(np.array(iris_data)[:, 1:].astype(np.float32))
        self.y = torch.from_numpy(np.array(iris_data)[:, [0]].astype(np.longlong).flatten())
        self.n_samples = self.x.shape[0]

    # working for indexing
    def __getitem__(self, index):
        
        return self.x[index], self.y[index]

    # return the length of our dataset
    def __len__(self):

        return self.n_samples


dataset = IrisDataset()

# Sample Dataset with Feature Size and Multiple Output Calssification

In [3]:
print(dataset[:5][0])

tensor([[5.1000, 3.5000, 1.4000, 0.2000],
        [4.9000, 3.0000, 1.4000, 0.2000],
        [4.7000, 3.2000, 1.3000, 0.2000],
        [4.6000, 3.1000, 1.5000, 0.2000],
        [5.0000, 3.6000, 1.4000, 0.2000]])


In [4]:
dataset[:,][1]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2])

# Train-Test Split and Build Dataloader

In [38]:
Split_Size = 0.8
train_size = int(Split_Size * len(dataset))
test_size  = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])


batch_size = 20
train_dataloader = DataLoader(dataset=train_dataset , batch_size=batch_size)
test_dataloader  = DataLoader(dataset=test_dataset  , batch_size=batch_size)

30

In [6]:
for i,(data,label) in enumerate(train_dataloader):
    print(f"iteration ({i+1})" + ", X_train Size :",data.size() , ", Y_train Size :",label.size())

iteration (1), X_train Size : torch.Size([20, 4]) , Y_train Size : torch.Size([20])
iteration (2), X_train Size : torch.Size([20, 4]) , Y_train Size : torch.Size([20])
iteration (3), X_train Size : torch.Size([20, 4]) , Y_train Size : torch.Size([20])
iteration (4), X_train Size : torch.Size([20, 4]) , Y_train Size : torch.Size([20])
iteration (5), X_train Size : torch.Size([20, 4]) , Y_train Size : torch.Size([20])
iteration (6), X_train Size : torch.Size([20, 4]) , Y_train Size : torch.Size([20])


# Build a Simple Neural Network Model

In [7]:
class Feed_Forward_Neural_Net(nn.Module):
    
    """
    input_dim --> regression features
    """
    
    def __init__(self,input_size, hidden_size, num_classes):
        super(Feed_Forward_Neural_Net, self).__init__()

        self.linear_1  = nn.Linear(input_size,hidden_size)  
        self.relu      = nn.ReLU()
        self.linear_2  = nn.Linear(hidden_size,num_classes)

    def forward(self, x):

        out = self.linear_1(x)
        out = self.relu(out)
        out = self.linear_2(out)
        
        return out

# Training Parameters

In [14]:
class parameters():

    def __init__(self) :

        # Layer Neuron Setting
        self.input_size    = 4
        self.hidden_size   = 10
        self.num_classes   = 3
        
        # Epochs and Learning Rate 
        self.num_epochs    = 20
        self.learning_rate = 0.01

        # Cross Entropy
        self.criterion     = nn.CrossEntropyLoss()


parameter = parameters()

# adam algorithm
model     = Feed_Forward_Neural_Net(parameter.input_size, parameter.hidden_size, parameter.num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=parameter.learning_rate,weight_decay=0.001)

# Start Training

In [15]:
for epoch in range(parameter.num_epochs):

    for i, (datas, labels) in enumerate(train_dataloader):
        
        # init optimizer
        optimizer.zero_grad()
        
        # forward -> backward -> update
        outputs = model(datas)
        loss    = parameter.criterion(outputs, labels)
        loss.backward()

        optimizer.step()

    if epoch % 2 == 0 :
        print(f'epoch {epoch+2}/{parameter.num_epochs} , loss = {loss.item():.4f}')

epoch 2/20 , loss = 0.8755
epoch 4/20 , loss = 0.6234
epoch 6/20 , loss = 0.4645
epoch 8/20 , loss = 0.3813
epoch 10/20 , loss = 0.3230
epoch 12/20 , loss = 0.2780
epoch 14/20 , loss = 0.2372
epoch 16/20 , loss = 0.2007
epoch 18/20 , loss = 0.1695
epoch 20/20 , loss = 0.1431


# Test Dataset

In [45]:
from sklearn.metrics import classification_report

outputs            = model(test_dataset[:][0].float())
heck , predictions = torch.max(outputs, 1)
test_label         = test_dataset[:][1].float()

print( classification_report(predictions,test_label) )

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        11
           1       0.91      1.00      0.95        10
           2       1.00      0.89      0.94         9

    accuracy                           0.97        30
   macro avg       0.97      0.96      0.96        30
weighted avg       0.97      0.97      0.97        30



In [46]:
with torch.no_grad():

    n_correct = 0
    n_samples = 0

    for datas, labels in test_dataloader:

        outputs = model(datas.float())
        check , predictions = torch.max(outputs, 1)

        n_samples += labels.shape[0]
        n_correct += (predictions == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'accuracy = {np.round(acc,decimals=2)} %')

accuracy = 96.67 %
